Copyright 2023 The TensorFlow Authors.


In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# CNN Daily データセットを使った大規模言語モデルの TFX Pipeline チュートリアル

この codelab では、KerasNLP を使用して、GPT-2 モデルという事前トレーニング済みの大規模言語モデル（LLM）を読み込み、データセットに合わせてファインチューニングを行います。このデモで使用されるデータセットは、CNN daily データセットです。ここでは、エンドツーエンドのプロセスを実演する目的のみで GPT-2 が使用されています。この codelab で紹介されるテクニックとツールは、Google T5 などの他のジェネレーティブ言語モデルに移植できる可能性があります。

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/ja/tfx/tutorials/tfx/CSV_Downloader_Component"> <img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/CSV_Downloader_Component.ipynb"> <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/CSV_Downloader_Component.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tfx/tutorials/tfx/CSV_Downloader_Component.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table></div>

# 始める前に

Colab には様々な種類のランタイムが用意されています。**ランタイム-&gt; ランタイムのタイプの変更** にアクセスし、必ず GPU ハードウェアアクセラレータのランタイムを選択してください。GPT-2 モデルのファインチューニングを行うため、12G 以上のシステム RAM と約 15G の GPU RAM が必要です。

# セットアップ

まず、TFX Python パッケージをインストールします。

## Pip をアップグレードする

ローカルで実行する場合にシステムの Pip をアップグレードしないように、Colab で実行していることを確認してください。もちろん、ローカルシステムは個別にアップグレードできます。

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

## TFX をインストールする

TFX では現在、Colab の Python 3.10 との問題が発生しています。したがって、以下のコマンド

```
!pip install -U tfx
```

を実行して tfx をインストールすると**失敗します**。そのため、以下のコードに従ってください。

In [ ]:
%%shell
update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 3
curl -O https://bootstrap.pypa.io/get-pip.py
python get-pip.py

In [ ]:
# 1) TFX relies on an old version of google-api-core so we let google-auth float
# for the install. We grep it out below:
!grep -v google-auth /etc/requirements.core.in > requirements.txt

# 2) httplib2 should be included in /etc/requirements.core.in but it's not for
# reasons. We ensure it's included:
!grep httplib2 /etc/requirements.user.in >> requirements.txt

# 3) google.colab package is not available as a wheel. We symlink that in so
# it's on the sys.path of Python 3.8:
!mkdir /usr/local/lib/python3.8/dist-packages/google
!ln -s /usr/local/lib/python3.10/dist-packages/google/colab /usr/local/lib/python3.8/dist-packages/google/colab

# Now with those pre-requisites out of the way:
!pip install tfx==1.13.0 -r requirements.txt

In [ ]:
!pip install keras_nlp

# インポート

まず、インポートを片付けてしまいましょう。

In [ ]:
from tensorflow import keras
from tfx.types import Channel
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

## shapely をアンインストールする

TODO(b/263441833) これは、ImportError を回避するための一時的な策です。最終的には、追加の依存関係をアンインストールする代わりに Bigquery の最近のバージョンをサポートすることで処理される必要があります。


In [ ]:
!pip uninstall shapely -y

## ランタイムを再起動しましたか？

Google Colab を使用している場合は、上記のセルを初めて実行した後に、「ランタイムを再起動」ボタンをクリックするか、「ランタイム」&gt;「ランタイムの再起動...」メニューを使って、ランタイムを再起動する必要があります。これは、Colab がパッケージを読み込むために必要な作業です。

TensorFlow と TFX のバージョンを確認します。

ライブラリのバージョンを確認します。

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

## 変数をセットアップする

パイプラインを定義するために使用する変数がいくつかあります。これらの変数は、必要に応じてカスタマイズすることが可能です。デフォルトでは、パイプラインからのすべての出力は、現在のディレクトリに生成されます。

# CSV Downloader

パイプラインの効率を高め、自動化の可能性を得るために、ダウンロードされる CSV ファイルへのダウンロードリンクを取得するコンポーネントを使用すると便利です。さらに、TFX 本番 ML パイプラインには、パイプラインコンポーネント、実行、および生成されるアーティファクトの情報を含むメタデータを収集するという重要な目標があります。言い換えると、メタデータの目的は、パイプラインコンポーネントの系統を解析して問題をデバッグすることであるため、CSV Downloader コンポーネントを使うことで、データのソースと、データがパイプラインに進入するまえに行われた前処理ステップについての情報をログ記録し、追跡することができます。このセクションでは、CSVdoc という新しいアーティファクトを宣言し、データセットの情報を保管して CSVdoc アーティファクトの URI にある CSV Fairuwo ダウンロードする CSV Downloader というカスタムコンポーネントを作成します。

In [ ]:
from tfx.types import artifact
from tfx import types

Property = artifact.Property
PropertyType = artifact.PropertyType

URL_PROPERTY = Property(type=PropertyType.STRING)
PATH_PROPERTY = Property(type=PropertyType.STRING)

class CsvDoc(types.Artifact):
  """ Artifact that contains the CSV dataset.

     - 'url' : saves the source of the original data.
     - 'path': saves the path to the CSV file.
  """

  TYPE_NAME = 'CsvDoc'
  PROPERTIES = {
      'url' : URL_PROPERTY,
      'path': PATH_PROPERTY,
  }

In [ ]:
from absl import logging
import requests
import os
import tfx.v1 as tfx
from tfx.dsl.component.experimental.decorators import component

@tfx.dsl.components.component
def CsvDownloaderComponent(
    url: tfx.dsl.components.Parameter[str],
    file_name: tfx.dsl.components.Parameter[str],
    saved_file: tfx.dsl.components.OutputArtifact[CsvDoc],
) -> None:
  response = requests.get(url)
  saved_file.url = url
  if response.status_code == 200:
    file_path = os.path.join(saved_file.uri, file_name)
    saved_file.path = file_path
    url_content = response.content
    with open(file_path, 'wb') as csv_file:
      csv_file.write(url_content)
    logging.info(f"CSV file saved successfully at {file_path}")
  else:
    raise Exception("CSV file failed to be saved.")

In [ ]:
downloader = CsvDownloaderComponent(
  url = 'https://drive.google.com/uc?id=1YdZsJlRafqxiNSl0nHQkwR7rzrNlN9LI&export=download', file_name ='testing_doc.csv')

In [ ]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext()

In [ ]:
context.run(downloader, enable_cache = False)